In [31]:
import tensorflow as tf
tf.reset_default_graph()

In [32]:
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.ops import rnn, rnn_cell_impl
mnist = input_data.read_data_sets("./tmp/data/", one_hot = True)

Extracting ./tmp/data/train-images-idx3-ubyte.gz
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz


In [33]:
epochs = 10
n_classes = 10
batch_size = 128

In [34]:
chunk_size = 28
n_chunks = 28
rnn_size = 256

In [35]:
x = tf.placeholder(tf.float32, [None, n_chunks, chunk_size])
y = tf.placeholder(tf.float32)

In [36]:
def recurrent_neural_network(x):
    layer = {'weights': tf.Variable(tf.truncated_normal([rnn_size, n_classes], stddev=0.1)), 
             'biases': tf.Variable(tf.truncated_normal([n_classes], stddev=0.1))}
    
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    outputs, states = tf.contrib.rnn.static_rnn(lstm, x, dtype=tf.float32)
    
    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']
    return output

In [37]:
def train_neural_nextwork(x):
    prediction = recurrent_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size, n_chunks, chunk_size))
                
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                
                epoch_loss += c
            
            print('Epoch', epoch, 'completed out of', epochs, 'loss:', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        
        print('Accuracy:', accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))

In [38]:
train_neural_nextwork(x)

Epoch 0 completed out of 10 loss: 224.208552513
Epoch 1 completed out of 10 loss: 57.5267912392
Epoch 2 completed out of 10 loss: 40.4369356995
Epoch 3 completed out of 10 loss: 28.8354433868
Epoch 4 completed out of 10 loss: 25.0495018652
Epoch 5 completed out of 10 loss: 19.621191748
Epoch 6 completed out of 10 loss: 17.7952457049
Epoch 7 completed out of 10 loss: 14.8750839079
Epoch 8 completed out of 10 loss: 13.3758171211
Epoch 9 completed out of 10 loss: 11.6183907114
Accuracy: 0.9855
